# "Hi, We're Called Atmosphere!"

**First scrape the lyrics**

In [1]:
import api_key

In [2]:
client_access_token = api_key.your_client_access_token

In [3]:
from bs4 import BeautifulSoup
import re
import lyricsgenius
LyricsGenius = lyricsgenius.Genius(client_access_token)
import requests
import os
from pathlib import Path
import glob

**RegEx Function to Clean Songs**

In [166]:
def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    return clean_song_title

In [167]:
def clean_album(album):
    album = album.replace(" ", "-")
    album = album.replace(":", "")
    album = album.replace("(", "-")
    album = album.replace(")", "")
    album = album.replace("'", "-")
    album = album.replace(",", "")
    return album

In [162]:
clean_album('To All My Friends, Blood Makes the Blade Holy: The Atmosphere Eps')

'To-All-My-Friends-Blood-Makes-the-Blade-Holy-The-Atmosphere-Eps'

**Make Function to Scrape Song Titles for Album**

In [164]:
def get_all_songs_from_album(artist, album):
    
    artist = artist.replace(" ", "-")
    album = clean_album(album)
    
    response = requests.get(f"https://genius.com/albums/{artist}/{album}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
        
    return clean_songs

In [168]:
get_all_songs_from_album("Atmosphere", 'Whenever')

['Bde Maka Ska',
 'Push Play',
 'Postal Lady',
 'Love Each Other',
 'Romance',
 'Dearly Beloved',
 'The Hands of Time',
 'Whenever',
 'Lovely',
 'Son of Abyss',
 "You're Gonna Go",
 'The Ceiling']

**Make Function to Download Lyrics For All Songs in Album**

In [ ]:
def download_album_lyrics(artist, album): 
    
    # Set up LyricsGenius with your Genius API client access token
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    # With the function that we previously created, go to Genius.com and get all song titles for a particular artist's album
    clean_songs = get_all_songs_from_album(artist, album)
    
    for song in clean_songs:
        
        #For each song in the list, search for that song with LyricsGenius
        song_object = LyricsGenius.search_song(song, artist)
        
        #If the song is not empty
        if song_object != None:
            
            #Do some cleaning and prep for the filename of the song
            artist_title = artist.replace(" ", "-")
            album_title = clean_album(album)
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            
            #Establish the filename for each song inside a directory that begins with the artist's name and album title
            #added the ../ to move it out of the notebooks folder because maybe that will work
            filename=f"../All/{artist_title}-{album_title}/{song_title}"
            
            #A line of code that we need to create a directory
             #os.makedirs(os.path.dirname(filename), exist_ok=True)
            Path(f"../All/{artist_title}-{album_title}").mkdir(parents=True, exist_ok=True)
            
            #Save the lyrics for the song as a text file
            song_object.save_lyrics(filename=filename, extension='txt', sanitize = False)
            #uncomment below if you want to check if things are matching up
            #title= f"{filename}.txt"
            #print(title)
        #If the song doesn't contain lyrics
        else:
            print('No lyrics')

In [ ]:
download_album_lyrics('Atmosphere', 'Whenever')

**Use glob to combine all lyrics for an album into one**

In [ ]:
def glob_songs(artist, album):
    artist = artist.replace(" ", "-")
    album = clean_album(album)
    #running the download function
    download_album_lyrics(artist, album)
    
    directory = f"../All/{artist}-{album}/"
    #print(directory)
    files = glob.glob(f"{directory}/*.txt")
    return files

**Now that the songs are globed, combine them into one text file for the whole album**

In [ ]:
def create_one_album_file(artist, album):
    #clean up the inputs
    artist = artist.replace(" ", "-")
    album = clean_album(album)
    #use the glob function we made
    files = glob_songs(artist, album)
    
    #create a master directory for the artist
    Path(f"../{artist}").mkdir(parents=True, exist_ok=True)
    
    with open(f"../{artist}/{album}.txt", "wb") as outfile:
        for file in files:
            with open(file, "rb") as infile:
                outfile.write(infile.read())

In [ ]:
create_one_album_file('Atmosphere', 'Whenever')

In [ ]:
create_one_album_file('Atmosphere', 'Mi Vida Local')

In [ ]:
create_one_album_file('Atmosphere', 'Fishing Blues')

In [ ]:
create_one_album_file('Atmosphere', 'Frida Kahlo vs Ezra Pound')

In [ ]:
create_one_album_file('Atmosphere', 'Southsiders')

In [ ]:
create_one_album_file('Atmosphere', 'The Family Sign')

In [ ]:
create_one_album_file('Atmosphere', 'To All My Friends, Blood Makes the Blade Holy: The Atmosphere Eps')

In [ ]:
create_one_album_file('Atmosphere', 'When Life Gives You Lemons, You Paint That Shit Gold')

In [ ]:
create_one_album_file('Atmosphere', "You Can't Imagine How Much Fun We're Having")

In [ ]:
create_one_album_file('Atmosphere', 'Headshots: Se7en')

In [ ]:
create_one_album_file('Atmosphere', "Seven's Travels")

In [ ]:
create_one_album_file('Atmosphere', "God Loves Ugly")

In [ ]:
create_one_album_file('Atmosphere', "Lucy Ford: The Atmosphere EP's")

In [ ]:
create_one_album_file('Atmosphere', "Overcast")

**Special code Sad Clown Bad Year:**

In [ ]:
#This is to download the four Sad Clown Bad Dub Albums (#9-12) that are included on Sad Clown Bad Year
def download_sad_clown(artist, album): 
    
    # Set up LyricsGenius with your Genius API client access token
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    # With the function that we previously created, go to Genius.com and get all song titles for a particular artist's album
    clean_songs = get_all_songs_from_album(artist, album)
    
    for song in clean_songs:
        
        #For each song in the list, search for that song with LyricsGenius
        song_object = LyricsGenius.search_song(song, artist)
        
        #If the song is not empty
        if song_object != None:
            
            #Do some cleaning and prep for the filename of the song
            artist_title = artist.replace(" ", "-")
            album_title = clean_album(album)
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            
            #Establish the filename for each song inside a directory that begins with the artist's name and album title
            #added the ../ to move it out of the notebooks folder because maybe that will work
            filename=f"../All/{artist_title}-Sad-Clown-Bad-Year/{song_title}"
            
            #A line of code that we need to create a directory
             #os.makedirs(os.path.dirname(filename), exist_ok=True)
            Path(f"../All/{artist_title}-Sad-Clown-Bad-Year").mkdir(parents=True, exist_ok=True)
            
            #Save the lyrics for the song as a text file
            song_object.save_lyrics(filename=filename, extension='txt', sanitize = False)
            #uncomment below if you want to check if things are matching up
            #title= f"{filename}.txt"
            #print(title)
        #If the song doesn't contain lyrics
        else:
            print('No lyrics')

In [ ]:
download_sad_clown('Atmosphere', "Sad Clown Bad Summer 9")

In [ ]:
download_sad_clown('Atmosphere', 'Sad Clown Bad Fall 10')

In [ ]:
download_sad_clown('Atmosphere', 'Sad Clown Bad Winter 11')

In [ ]:
download_sad_clown('Atmosphere', 'Sad Clown Bad Spring 12')

In [ ]:
#only run this after running download_sad_clown for all four dubs
def glob_sad_clown(artist):
    directory = f"../All/{artist}-Sad-Clown-Bad-Year/"
    #print(directory)
    files = glob.glob(f"{directory}/*.txt")
    return files

In [ ]:
#this is the same as create_one_album_file but made specifically for Sad Clown Bad Dubs
#make sure you have all files into Sad-Clown-Bad-Year first!
def create_sad_clown(artist, album):
    #clean up the inputs
    artist = artist.replace(" ", "-")
    album = clean_album(album)
    #use the glob function we made
    files = glob_sad_clown(artist)
    
    #create a master directory for the artist
    Path(f"../{artist}").mkdir(parents=True, exist_ok=True)
    
    with open(f"../{artist}/{album}.txt", "wb") as outfile:
        for file in files:
            with open(file, "rb") as infile:
                outfile.write(infile.read())

In [ ]:
create_sad_clown('Atmosphere', 'Sad Clown Bad Year')